### This code aims at converting tensor back to text data 

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import torch
import json
import collections
from pytorch_pretrained_bert.tokenization import BertTokenizer
import string
import copy

In [3]:
tokenizer = BertTokenizer.from_pretrained('./saved_models/gaku_essay_55/', do_lower_case=True)

In [4]:
base = '/nfs/nas-5.1/kyhuang/preprocess/cmv_raw_origin_v4/heldout/'

In [5]:
pred = {}
index_mapping = {}

for side, key in enumerate(['pos', 'neg', 'op']):
    pred[key] = torch.load('./pred_result/origin_v4/heldout/gaku_essay_{}'.format(key))
    with open('./mapping/origin_v4/heldout/mapping_clean_{}'.format(key)) as f:
        index_mapping[key] = [line for line in f]
    

In [6]:
pred_type = copy.deepcopy(pred)

In [7]:
def checkandvote(sent, recovers, bios, dtypes):
    mapping_bio = ['B', 'I', 'O', 'E']
    mapping_type = ['P', 'C']
    
    
    recover_sent, recover_bio = [], []
    adu_type = []
    count = [0, 0]
    
    last = torch.rand(2)
    for index, (word, r, bio, dtype) in enumerate(zip(sent[1:], recovers, bios, dtypes)):
        if(word[:2]=='##'):
            if(bio==0):
                last = torch.zeros_like(dtype)
                
                recover_bio[-1] = 'B'
                count[0] += 1
                
            if( bio == 3 ):
                if( recover_bio[-1] == 'B' ):
                    recover_bio[-1] = 'O'
                    count[0] -= 1
                else:
                    adu_type.append(
                        mapping_type[last.argmax(-1).item()]
                    )
                    recover_bio[-1] = mapping_bio[bio]
                    count[1] += 1
                
            recover_sent[-1] += word.split('#')[-1]
        else:
            if( bio == 0 ):
                last = torch.zeros_like(dtype)
                count[0] += 1
                
            elif( bio == 3 ):
                adu_type.append(
                    mapping_type[last.argmax(-1).item()]
                )
                count[1] += 1

            recover_sent.append(word)
            recover_bio.append(mapping_bio[bio])

        if(bio<=1):
            last += dtype
    
    if(count[1] != count[0]):
        adu_type.append(
            mapping_type[last.argmax(-1).item()]
        )
        count[1] +=1
        recover_bio[-1] = 'O'
        if(recover_bio[-2] == 'B'):
            recover_bio[-2] = 'O'
        else:
            recover_bio[-2] = 'E'
        
    assert count[0]==(len(adu_type)),"{}\n{}\n{}".format(bios, recover_bio, adu_type)
    assert count[1]==(len(adu_type)),"{}\n{}\n{}".format(bios, recover_bio, adu_type)
    return recover_sent, recover_bio, adu_type

In [8]:
recover = {}
for key in ['pos', 'neg', 'op']:
    recover[key] = {'sent':[], 'bio':[], 'index':[],'type':[]}
    for index in range(len(pred[key]['id'])):
        temp = checkandvote(
                        tokenizer.convert_ids_to_tokens(pred[key]['id'][index].tolist()), 
                        pred[key]['recover'][index].tolist(), pred[key]['bio'][index],
                        torch.tensor(pred_type[key]['type'][index]).softmax(-1)
                       )
        
        recover[key]['sent'].append(' '.join(temp[0]))
        recover[key]['bio'].append(temp[1])
        recover[key]['type'].append(temp[2])
        recover[key]['index'].append(pred[key]['index'][index].item())
        
        

In [9]:
temp_recover = copy.deepcopy(recover)

## some check function

In [17]:
def printorigin(side, index):
    for word, r, bio in zip(tokenizer.convert_ids_to_tokens(pred[side]['id'][index].tolist()[1:]), pred[side]['recover'][index][:-2], pred[side]['bio'][index][:-2]):
        print(bio, r, word)

In [16]:
def printrecover(recover, side, index):
    print(recover[side]['type'][index])
    for word, bio in zip(recover[side]['sent'][index].split(),recover[side]['bio'][index]):
        print(word, bio)

In [23]:
# find next token is that
def checkandcollect(recover, key):
    rm_stat = collections.defaultdict(int)

    for side in recover:
        for r_index in range(len(recover[side]['bio'])):
            buffer = []
            sent = recover[side]['sent'][r_index].split()
            bios = recover[side]['bio'][r_index]
            
            for index, (word, bio) in enumerate( zip(sent[:-1], bios) ):
                if( bio =='I' and word =='because'):
                    print(sent)
                    print(bios)
                    None +1
                    
    return rm_stat

In [24]:
def checkkey(recover, key):
    for side in recover:
        for r_index in range(len(recover[side]['bio'])):
            buffer = []
            sent = recover[side]['sent'][r_index].split()
            bios = recover[side]['bio'][r_index]

            for index, (word, bio) in enumerate( zip(sent[:-1], bios) ):
                if( bio =='I' and word ==key):
                    print(' '.join(sent))
                    print(sent[max(-24+index,0):index+8])
                    print(bios[max(-24+index,0):index+8])
                    None +1

In [12]:
rm_sent = {}
rm_sent['that'] = checkandcollect(recover, 'that')
rm_sent['whether'] = checkandcollect(recover, 'whether')
rm_sent['if'] = checkandcollect(recover, 'if')
rm_sent['is'] = checkandcollect(recover, 'is')
rm_sent['are'] = checkandcollect(recover, 'are')
rm_sent['which'] = checkandcollect(recover, 'which')

['>', 'thanks', 'for', 'the', 'kind', 'words', '.', 'i', 'appreciate', 'it', '.', 'i', 'also', 'share', 'your', 'concern', 'for', 'our', 'country', ',', 'and', 'agree', 'wholeheartedly', 'that', 'every', 'vote', 'counts', '.', 'however', ',', 'i', "'", 'm', 'afraid', 'i', 'can', "'", 't', 'encourage', 'millions', 'of', 'people', 'whom', 'i', "'", 've', 'never', 'met', 'to', 'just', 'run', 'out', 'and', 'cast', 'a', 'ballot', ',', 'simply', 'because', 'they', 'have', 'the', 'right', 'to', 'vote', '.', 'that', 'would', 'be', 'like', 'encouraging', 'everyone', 'to', 'buy', 'an', 'ar', '-', '15', ',', 'simply', 'because', 'they', 'have', 'the', 'right', 'to', 'bear', 'arms', '.', 'i', 'would', 'need', 'to', 'know', 'a', 'few', 'things', 'about', 'them', 'before', 'offering', 'that', 'kind', 'of', 'encouragement', '.', 'for', 'instance', ',', 'do', 'they', 'know', 'how', 'to', 'care', 'for', 'a', 'weapon', '?', 'can', 'they', 'afford', 'the', 'cost', 'of', 'the', 'weapon', '?', 'do', 'they'

TypeError: unsupported operand type(s) for +: 'NoneType' and 'int'

In [ ]:
def checkandprint(check, arr, windex, sent, bios):
    key = 'because of'
    l = len(key.split())
    c = ' '.join(arr[:l])
    
    if(c == key):
        print()
        print(arr)
        print(bios[windex:windex+len(arr)])
        
        for i in range(1, min(len(arr), 6)):
            check[' '.join(arr[:i])] += 1
            
        return True
    
    return False

check = collections.defaultdict(int)
length = 8
c = 0
for side in range(2):
    for index in range(len(recover[side]['sent'])):
        buffer = []
        sent = temp_recover[side]['sent'][index].split()
        bios =  temp_recover[side]['bio'][index]
        for windex, (word, bio) in enumerate(zip(sent, bios)):
            if(bio != 'O'):
                buffer.append(word)
                
                if( bio == 'E' ):
                    if(checkandprint(check, buffer, windex-len(buffer)+1, sent, bios)):
                        print(side, index, windex)
                        c += 1
                    buffer = []
c                    

In [ ]:
#sorted([(key, val) for key, val in check.items()], key=lambda x:x[1], reverse=True)

In [80]:
length = 8
stat = []
for _ in range(2):
    stat.append([collections.defaultdict(int) for _ in range(length)])
    
def adddict(dataset, words, length, front=True):
    l = min(len(buffer), length)
    if(front):
        dataset[l-1][' '.join(buffer[:l])] += 1
    else:
        dataset[l-1][' '.join(buffer[-l:])] += 1
    
for side in range(2):
    for index in range(len(recover[side]['sent'])):
        buffer = []
        for word, bio in zip(recover[side]['sent'][index].split(),recover[side]['bio'][index]):
            if(bio == 'B' or bio == 'I'):
                buffer.append(word)
                                        
            if(bio == 'E'):
                buffer.append(word)
                #if(len(buffer) == 2):
                #    print(side, index)
                adddict(stat[0], buffer, length, True)
                adddict(stat[1], buffer, length, False)

for i in range(2):
    for j in range(length):
        stat[i][j] = sorted([(key, value) for key, value in stat[i][j].items()], key = lambda x:x[1], reverse=True)

In [114]:
for _ in stat[0][:6]:
    for key, val in _:
        if('problem' in key):
            print(val, key)

3 this is problematic
1 a major problem
1 lots of problems
1 thats the problem
1 there are problems
1 both are problems
1 it becomes problematic
1 they are problematic
23 this is the problem
19 this is a problem
6 that is a problem
4 problems with mars 1
4 this is my problem
4 these problems are necessary
4 it is a problem
4 my problem with it
3 i have a problem
3 overpopulation is a problem
2 i have this problem
2 obesity causes health problems
2 this is your problem
2 that is their problem
2 one of the problems
2 there are several problems
2 the problem is regional
2 the problem was ricky
2 there are other problems
2 inflation is a problem
2 the problem is support
2 radicalization is a problem
2 speceisism is inherently problematic
2 the problems were addressed
1 the problem is risk
1 i have memory problems
1 the problem with violence
1 the problem is huge
1 another problem is space
1 trump represents the problem
1 it is incredibly problematic
1 which is my problem
1 not even a probl

In [184]:
stat_word = [collections.defaultdict(int), collections.defaultdict(int)]

for side in range(2):
    for index in range(len(recover[side]['sent'])):
        for word, bio in zip(recover[side]['sent'][index].split(),recover[side]['bio'][index]):
            if(bio == 'B'):
                stat_word[0][word] += 1
            elif(bio == 'E'):
                stat_word[1][word] += 1

sort_stat_word = [None, None]
for i in range(2):
    sort_stat_word[i] = sorted([(key, val) for key, val in stat_word[i].items()],key= lambda x:x[1], reverse=True)

## postprocess

In [10]:
def remove_fe_punctation(sent, bios, dtypes):
    #print(sent)
    #print(bios)
    punc = '!#$%&\*+,-./:;?@^_`|~'
    
    new_type = []
    type_index = 0
    
    for index in range(len(sent)-1):
        if(bios[index] == 'B'):
            if(sent[index] in punc):
                bios[index] = 'O'
                if(bios[index+1] == 'E'):
                    bios[index+1] = 'O'
                    type_index += 1
                else:
                    bios[index+1] = 'B'
        elif(bios[index] == 'E'):
            new_type.append( dtypes[type_index])
            type_index += 1
    else:
        if(bios[len(sent)-1] == 'E'):
            new_type.append( dtypes[type_index])
    # return sent, bios, new_type
    
    new_new_type = []
    type_index = len(new_type)-1
    for index in range(len(sent)-1, 0, -1):
        if(bios[index] == 'E'):
            if(sent[index] in punc):
                bios[index] = 'O'
                if(bios[index-1] == 'B'):
                    bios[index-1] = 'O'
                    type_index -= 1
                else:
                    bios[index-1] = 'E'
        elif(bios[index] == 'B'):
            new_new_type.append( new_type[type_index])
            type_index -= 1
    else:
        if(bios[0] == 'B'):
            new_new_type.append( new_type[type_index])
                
    return ' '.join(sent), bios, new_new_type

In [11]:
def remove_last_conj(sent, bios, dtypes):
    #print(sent)
    #print(bios)
    #print(dtypes)
    conj = ['and', 'or']
    
    new_type = []
    type_index = len(dtypes)-1
    
    for index in range(len(sent)-1, 0, -1):
        if(bios[index] == 'E'):
            if(sent[index] in conj):
                bios[index] = 'O'
                if(bios[index-1] == 'B'):
                    bios[index-1] = 'O'
                    type_index -= 1
                else:
                    bios[index-1] = 'E'
        elif(bios[index] == 'B'):
            new_type.append( dtypes[type_index])
            type_index -= 1
    else:
        if(bios[0] == 'B'):
            #print(type_index)
            new_type.append( dtypes[type_index])
    
    return ' '.join(sent), bios, new_type[::-1]

In [12]:
def removebio(sent, bios, dtype, remove_set):
    new_sent, new_bio, new_type = [], [], []
    buffer = []
    type_index = 0
            
    for index, (word, bio) in enumerate( zip(sent, bios)):
        new_sent.append( word )

        if( bio=='O'):
            new_bio.append( bio )
        else:
            buffer.append(word)

            if( bio == 'E'):
                l = len(buffer)
                new_type.append( dtype[type_index] )
                type_index += 1
                
                for remove_len in sorted(sort_front_conj, reverse=True):
                    if(len(buffer)<remove_len):
                        continue
                        
                    remove_sents = sort_front_conj[remove_len]
                    check_sent = ' '.join(buffer[:remove_len])
                

                    if(check_sent in remove_sents):
                        # convert these data to others
                        new_bio.extend( ['O']*(remove_len) )
                        if((l-remove_len)>1):
                            l = l-remove_len
                            new_bio.append('B')
                            if(l>=2):
                                new_bio.extend( ['I']*(l-2) )
                            new_bio.append('E')
                        elif( (l-remove_len)==1 ):
                            new_bio.append('O')
                            new_type.pop()
                        elif(l==remove_len):
                            new_type.pop()
                        else:
                            raise ValueError(' should be shorter')
                        break
                else:    
                    new_bio.append('B')
                    if(l>=2):
                        new_bio.extend( ['I']*(l-2) )
                    new_bio.append('E')

                buffer = []
    assert(len(new_sent) == len(new_bio)),"{}_{}\n{}\n{}\n".format(len(new_sent), len(new_bio), new_sent, new_bio)
    return ' '.join(new_sent), new_bio, new_type

In [13]:
recover = copy.deepcopy(temp_recover)

In [14]:
front_conj = json.load(open('qq.json'))
sort_front_conj = collections.defaultdict(list)

for _ in front_conj:
    if(_[0] in string.punctuation):
        continue
    l = len(_.strip().split())
    sort_front_conj[l].append(_)

In [15]:
for side in recover:
    for index in range(len(recover[side]['sent'])):
        temp = remove_fe_punctation(recover[side]['sent'][index].split().copy(), recover[side]['bio'][index].copy(), recover[side]['type'][index].copy())
        temp = remove_last_conj(temp[0].split().copy(), temp[1].copy(), temp[2].copy())
        temp = remove_fe_punctation(temp[0].split().copy(), temp[1].copy(), temp[2].copy())
        temp = removebio(temp[0].split().copy(), temp[1].copy(), temp[2].copy(), sort_front_conj)
        temp = remove_fe_punctation(temp[0].split().copy(), temp[1].copy(), temp[2].copy())
        
        recover[side]['sent'][index] = temp[0] 
        recover[side]['bio'][index]  = temp[1]
        recover[side]['type'][index] = temp[2]

In [16]:
for side in recover:
    for index in range(len(recover[side]['sent'])):
        sent = recover[side]['sent'][index].split()
        bios = recover[side]['bio'][index]
        dtype = recover[side]['type'][index]
        
        c = [0, 0]
        for word, bio in zip(sent, bios):
            if(bio == 'B'):
                c[0] += 1
            if(bio == 'E'):
                c[1] += 1
        assert(c[0] == c[1])
        assert(c[0] == len(dtype))

In [17]:
#stat = [collections.defaultdict(int), collections.defaultdict(int), collections.defaultdict(int), collections.defaultdict(int)]
for side in recover:
    for index in range(len(recover[side]['sent'])):
        if(('< link >' in recover[side]['sent']) or ('< cite >' in recover[side]['sent']) or ('< user >' in recover[side]['sent']) or ('< reply >' in recover[side]['sent'])):
            sent = recover[side]['sent'][index].split()
            bios = recover[side]['bio'][index]
            
            temp_sent, temp_bio = [], []
            word_index = 0
            while(word_index<len(sent)):
                temp_sent.append(sent[word_index])
                temp_bio.append(bios[word_index])
                for key in ['link', 'cite', 'reply', 'user']:
                    if(sent[word_index:word_index+3]==['<', key, '>']):
                        temp_sent.pop()
                        temp_bio.pop()
                        temp_sent.append('<{}>'.format(key))

                        if(bios[word_index:word_index+3] == ['I', 'I', 'I']):
                            temp_bio.append('I')
                        elif(bios[word_index:word_index+3] == ['I', 'I', 'E']):
                            temp_bio.append('E')
                        elif(bios[word_index:word_index+3] == ['I', 'E', 'O']):
                            temp_bio.append('E')

                        elif(bios[word_index:word_index+3] == ['E', 'O', 'O']):
                            temp_bio.append('E')

                        elif(bios[word_index:word_index+3] == ['B', 'I', 'I']):
                            temp_bio.append('B')
                        elif(bios[word_index:word_index+3] == ['B', 'I', 'E']):
                            temp_bio.append('O')

                        elif(bios[word_index:word_index+3] == ['O', 'O', 'O']):
                            temp_bio.append('O')
                        elif(bios[word_index:word_index+3] == ['O', 'B', 'I']):
                            temp_bio.append('B')
                        word_index += 2
                word_index += 1
                
            recover[side]['sent'][index] = ' '.join(temp_sent)
            recover[side]['bio'][index] = temp_bio

### get the clean post data and prepare paragraph data

In [18]:
pairs = {}
for side, key in enumerate(['pos', 'neg']):
    with open(base+'/parsing/clean_{}'.format(key)) as f:
        f.readline()

        for line in f:
            clean_post = json.loads(line)
            index, post_index, reply_index = [int(_) for _ in clean_post['uid'].split('_')]

            if(index not in pairs):
                pairs[index] = {'content':[[], []]}

            while(len(pairs[index]['content'][side])<=post_index):
                pairs[index]['content'][side].append([])

            while(len(pairs[index]['content'][side][post_index])<=reply_index):
                pairs[index]['content'][side][post_index].append(None)

            pairs[index]['content'][side][post_index][reply_index] = {
                'bio':[None for _ in clean_post['context']],
                'context':clean_post['context'],
                'author':clean_post['author'],
                'type':[None for _ in clean_post['context']]
            }

with open(base+'/parsing/clean_op') as f:
    f.readline()
    for line in f:
        clean_post = json.loads(line)    

        index = int(clean_post['uid'])
        pairs[index]['op_info'] = {
                'bio':[None for _ in clean_post['context']],
                'context':clean_post['context'],
                'author':clean_post['author'],
                'type':[None for _ in clean_post['context']],
                'topic':clean_post['topic'],
                'pos_author':clean_post['pos_author'],
            }

In [19]:
len(pairs)

883

In [20]:
for side, key in enumerate(['pos', 'neg']):
    for index, sent, bio, dtype in zip(recover[key]['index'], recover[key]['sent'], recover[key]['bio'], recover[key]['type']):
        index, post_index, reply_index, para_index = [ int(_) for _ in index_mapping[key][index].split('_')]

        # bert tokenizer is different from nltk, thus need to use the new one
        pairs[index]['content'][side][post_index][reply_index]['context'][para_index] = sent
        pairs[index]['content'][side][post_index][reply_index]['bio'][para_index] = ' '.join(bio)
        pairs[index]['content'][side][post_index][reply_index]['type'][para_index] = dtype

key = 'op'
for index, sent, bio, dtype in zip(recover[key]['index'], recover[key]['sent'], recover[key]['bio'], recover[key]['type']):
    index, para_index = [ int(_) for _ in index_mapping[key][index].split('_')]

    # bert tokenizer is different from nltk, thus need to use the new one
    pairs[index]['op_info']['context'][para_index] = sent
    pairs[index]['op_info']['bio'][para_index] = ' '.join(bio)
    pairs[index]['op_info']['type'][para_index] = dtype

In [21]:
count = 0
for index in pairs:
    if((len(pairs[index]['content'][0])>0) and (len(pairs[index]['content'][1])>0)):
        count += 1
count

883

In [22]:
def concat(total_type):
    dtype = []
    for _ in total_type:
        dtype.extend(_)
    return dtype
count = 0
pair_index = list(pairs.keys())
for index in pair_index:
    try:
        pairs[index]['op_info']['type'] = concat(pairs[index]['op_info']['type'])
    except:
        count += 1
        del pairs[index]
        continue
    
    for side in range(2):
        temp_post = []
        pos_authors = []
        for post, pos_author in zip(pairs[index]['content'][side], pairs[index]['op_info']['pos_author'][side]):
            temp_reply = []
            
            for reply in post:
                try:
                    temp_reply.append({
                        'bio':reply['bio'],
                        'context':reply['context'],
                        'author':reply['author'],
                        'type':concat(reply['type'])
                    })
                except TypeError:
                    count += 1
                    break
            else:
                pos_authors.append(pos_author)
                temp_post.append(temp_reply)
        
        pairs[index]['op_info']['pos_author'][side] = pos_authors
        pairs[index]['content'][side] = temp_post
        if(len(pairs[index]['content'][side]) == 0):
            del pairs[index]
            break
count, len(pairs)

(30, 868)

In [23]:
pairs[index]['op_info'].keys()

dict_keys(['bio', 'context', 'author', 'type', 'topic', 'pos_author'])

## convert pairs into posts

In [24]:
posts = [[], []]
for index in pairs:
    for side in range(2):
        op_info = {}
        for key in ['bio', 'context', 'author', 'type']:
            op_info[key] = pairs[index]['op_info'][key]
            
        for post_index, post in enumerate(pairs[index]['content'][side]):
            temp = [copy.deepcopy(op_info)]
            temp.extend(post)
            
            posts[side].append({
                'content':temp,
                'topic':pairs[index]['op_info']['topic'],
                'uid':'{}_{}'.format(index, post_index),
                'pos_author': pairs[index]['op_info']['pos_author'][side][post_index]
            })
            

In [25]:
# save post
for i, key in enumerate(['pos', 'neg']):
    with open(base+'/tree/clean_post_{}.jsons'.format(key), 'w') as f:
        for _ in posts[i]:
            f.write(json.dumps(_))
            f.write('\n')
# save post
for i, key in enumerate(['pos', 'neg']):
    with open(base+'/tree_bert/clean_post_{}.jsons'.format(key), 'w') as f:
        for _ in posts[i]:
            f.write(json.dumps(_))
            f.write('\n')

In [26]:
c = 0
for side in range(2):
    for post in posts[side]:
        l = 0
        for _ in post['content']:
            l += len(_['type'])
        if(l<100):
            c+=1
print(c, 2*len(posts[0]))

4293 6078


## elmo tree version

In [169]:
# load posts if needed
posts = []
for i, key in enumerate(['pos', 'neg']):
    with open(base+'/tree/clean_post_{}.jsons'.format(key)) as f:
        posts.append([json.loads(line) for line in f])

In [111]:
list(range(26,25,-1))

[26]

In [176]:
# process link 
c = []
def find_shell(index, last, context, adu_index):
    #print(last, index)
    if(index==0):
        return (adu_index, 0, 0)
    for i in range(index, last, -1):
        if(context[i] in [".", "!", "?", "</ac>", "<para>"]):
            if(i>=index):
                return (adu_index, index, index)
            else:
                return (adu_index, i+1, index)
    
    return (adu_index, index, index)


def prepare(data, elmo_preprocess):
    mask = []
    shell_span, span, elmo_index = [], [], []
    adu_label = []
    author = []
    para_author = []
    
    topic_elmo_index = len(elmo_preprocess)
    elmo_preprocess.append('<topic> '+data['topic']+' </topic>')
    elmo_preprocess[-1] = ' '.join(elmo_preprocess[-1].split())
    
    ac_position_info = []
    adu_index = 0
    for post_pos, post in enumerate(data['content']):
        for bio, context in zip(post['bio'], post['context']):
            elmo_index.append(len(elmo_preprocess))

            elmo_preprocess.append(['<para>'])
            bios, sent = bio.split(), context.split()

            last = -1
            for bio, word in zip(bios, sent):
                if(bio == 'B'):
                    span.append([adu_index, len(elmo_preprocess[-1]), 0])
                    shell_span.append(find_shell(len(elmo_preprocess[-1])-1, last, elmo_preprocess[-1], adu_index))
                    elmo_preprocess[-1].append('<ac>')
                    
                elmo_preprocess[-1].append(word)
                
                if(bio == 'E'):
                    span[-1][-1] = len(elmo_preprocess[-1])
                    last = span[-1][-1]
                    elmo_preprocess[-1].append('</ac>')
                    
            elmo_preprocess[-1].append('</para>')
            elmo_preprocess[-1] = ' '.join(elmo_preprocess[-1])
            # update parameter
            adu_index += 1
        # build mask
        dtype = post['type']
        prev_len, now_len = len(adu_label), len(dtype)
        for adu_pos, _ in enumerate(dtype):
            if(_ == 'P'):
                # constrain search space to this reply
                mask.append([0]*prev_len + [1]*now_len)

            elif(_ == 'C'):
                # constrain search space to all
                mask.append([1]*(prev_len + now_len))

            ac_position_info.append([adu_pos, post_pos])

        adu_label.extend( dtype )

    if(('[deleted]' in data['pos_author'][0]) and data['content'][-1]['author'] == '[deleted]'):
        for index, post in enumerate(data['content']):
            author.extend([1+index&1]*len(post['type']))
            para_author.extend([1+index&1]*len(post['bio']))
    else:
        for post in data['content']:
            if(post['author'] in data['pos_author'][0]):
                author.extend([1]*len(post['type']))
                para_author.extend([1]*len(post['bio']))
            elif(post['author'] in data['pos_author'][1]):
                author.extend([2]*len(post['type']))
                para_author.extend([2]*len(post['bio']))
            else:
                author.extend([0]*len(post['type']))
                para_author.extend([0]*len(post['bio']))

    adu_map = {'P':0, 'C':1}
    adu_label = [ adu_map[_] for _ in adu_label]
        
    return {
        'topic_index':topic_elmo_index, 
        'elmo_index':elmo_index,
        'shell_span':shell_span, 
        'span':span, 
        'mask':mask, 
        'author':author,
        'para_author':para_author,
        'adu_label':adu_label,
        'ac_position_info':ac_position_info
    }

In [178]:
prepare_data = [[], []]
elmo_preprocess = [[], []]

for side in range(2):
    for _ in posts[side]:
        temp = prepare(_, elmo_preprocess[side])
        temp['uid'] = _['uid']
        prepare_data[side].append(temp)

In [171]:
_

{'content': [{'bio': ['B I I I I I I I I I I I I I I I I I E O',
    'B I I I I I I I I I I I I I I I I I I I I I I I I I E O',
    'B I I I I I I I I I I I I I I I I I I I I I E O B I I I I I I I I I I I I E O O O O B I I I I I I I I I I I I I I I I E O B I I I I I I I I I I I I I I I I I I E O B I I I I I I E O',
    'B I I I I I I I I I I I I I I I I I I I I I I I I I E O O B I I I I I I I I I I I I I I I I I E O O O B I I I I I I I I I I I I I I I I I I I I I I I I I I I I E O B I I I I I I I I I I I I I I I I I I I E O',
    'O O O',
    'O'],
   'context': ['this is not meant as a disrespect to most organized religions , merely an observation i have come to .',
    'the widespread belief or following of most organized religions ( christianity , islam , judaism ) is a sign of human weakness in a couple of ways .',
    'the need for an afterlife in most religions satisfies our general fears of death , and the impermanence and futility of our lives . the theory is not backed by much

In [157]:
index, start, end = 4, 26, 28
index = prepare_data[0][0]['elmo_index'][index]
print(elmo_preprocess[0][index])
print()
print(' '.join(elmo_preprocess[0][index].split()[start:end+1]))

<para> <ac> the vice president often is called to act as a replacement for a duty or appearance if the president is not around </ac> . for instance , <ac> meeting foreign dignitaries if the president is out of town </ac> . <ac> there needs to be a second face of the executive branch </ac> . </para>

for instance ,


In [113]:
elmo_preprocess[0][prepare_data[0][0]['elmo_index'][1]].split()[26]

'.'

In [155]:
side, index = 0, 0
for a, b in zip(prepare_data[side][index]['shell_span'], prepare_data[side][index]['span']):
    print(a, b)

(0, 1, 5) [0, 6, 22]
(0, 23, 23) [0, 24, 34]
(0, 36, 37) [0, 38, 56]
(1, 0, 0) [1, 1, 25]
(1, 26, 26) [1, 27, 42]
(1, 43, 43) [1, 44, 52]
(1, 53, 53) [1, 54, 76]
(1, 78, 78) [1, 79, 99]
(1, 101, 107) [1, 108, 132]
(2, 1, 5) [2, 6, 31]
(2, 32, 32) [2, 33, 54]
(3, 0, 0) [3, 1, 38]
(3, 40, 40) [3, 41, 54]
(3, 68, 69) [3, 70, 99]
(3, 101, 102) [3, 103, 130]
(3, 132, 133) [3, 134, 149]
(3, 150, 150) [3, 151, 185]
(4, 0, 0) [4, 1, 24]
(4, 26, 28) [4, 29, 40]
(4, 41, 41) [4, 42, 54]
(5, 1, 3) [5, 4, 45]
(5, 46, 46) [5, 47, 60]
(6, 1, 15) [6, 16, 34]
(6, 35, 35) [6, 36, 56]
(6, 57, 58) [6, 59, 68]
(6, 70, 81) [6, 82, 104]
(6, 106, 109) [6, 110, 140]
(7, 0, 0) [7, 1, 12]
(7, 13, 13) [7, 14, 32]
(7, 33, 33) [7, 34, 67]
(7, 68, 68) [7, 69, 103]
(7, 105, 110) [7, 111, 136]
(7, 137, 137) [7, 138, 163]
(7, 165, 167) [7, 168, 192]
(7, 193, 193) [7, 194, 204]
(7, 205, 208) [7, 209, 222]
(8, 1, 2) [8, 3, 20]
(8, 21, 21) [8, 22, 48]
(8, 49, 49) [8, 50, 60]


In [33]:
posts[0][0]

{'content': [{'bio': ['O O O O O B I I I I I I I I I I I I I E O B I I I I I I I E O O O B I I I I I I I I I I I I I I I E O',
    'B I I I I I I I I I I I I I I I I I I I I I E O B I I I I I I I I I I I I E O B I I I I I E O B I I I I I I I I I I I I I I I I I I I E O O B I I I I I I I I I I I I I I I I I E O O O O O O O O B I I I I I I I I I I I I I I I I I I I I I E O',
    'O O O O O B I I I I I I I I I I I I I I I I I I I I I I E O B I I I I I I I I I I I I I I I I I I E O',
    'B I I I I I I I I I I I I I I I I I I I I I I I I I I I I I I I I I I E O O B I I I I I I I I I I E O O O O O O O O O O O O O O O B I I I I I I I I I I I I I I I I I I I I I I I I I I E O O O B I I I I I I I I I I I I I I I I I I I I I I I I E O O O B I I I I I I I I I I I I E O B I I I I I I I I I I I I I I I I I I I I I I I I I I I I I I I E O'],
   'context': ['from what i understand , the only significant duty given to the vice president is to preside over the senate ; even this has been shirked by pa

In [58]:
for side in range(2):
    for i, post in enumerate(prepare_data[side]):
        for _ in post['elmo_index']:
            assert(_<len(elmo_preprocess[side])),'{} {}'.format(side, i)
        assert(post['topic_index']<len(elmo_preprocess[side])),'{} {}'.format(side, i)

In [59]:
for side, key in enumerate(['pos', 'neg']):
    with open('./../preprocess/cmv_raw_v2/tree/cmv_elmo_{}.txt'.format(key), 'w') as f:
        for line in elmo_preprocess[side]:
            f.write(line)
            f.write('\n')

In [60]:
for i, key in enumerate(['pos', 'neg']):
    with open('./../preprocess/cmv_raw_v2/tree/pre_{}.jsons'.format(key), 'w') as f:
        for _ in prepare_data[i]:
            f.write(json.dumps(_))
            f.write('\n')

In [86]:
for side in range(2):
    for index, _ in enumerate(prepare_data[side]):
        if(0 in _['author']):
            print(side, index)

1 8
1 106
1 187
1 189
1 192
1 209
1 232
1 242
1 328
1 385
1 388
1 422
1 426
1 430
1 452
1 510
1 523
1 563
1 575
1 581
1 600
1 621
1 649
1 660
1 696
1 710
1 791
1 801
1 907
1 908
1 917
1 952
1 959
1 997
1 1042
1 1144
1 1185
1 1187
1 1189
1 1197
1 1208
1 1213
1 1225
1 1231
1 1247
1 1262
1 1297
1 1316
1 1331


In [97]:
side, index = 1, 106
print(posts[side][index]['pos_author'])
for _ in posts[side][index]['content']:
    print(_['author'], len(_['bio']), len(_['type']), end=' ')
temp = prepare(posts[side][index], elmo_preprocess[side])
print()
print(temp['author'])
print(temp['para_author'])

[['MicrosizeMe', 'gmoneygangster3'], ['DrinkyDrank', 'DangerouslyUnstable']]
MicrosizeMe 4 7 DrinkyDrank 1 2 scottevil110 3 4 mrgoodnighthairdo 1 1 DangerouslyUnstable 1 2 mrgoodnighthairdo 1 1 DangerouslyUnstable 2 8 ProfThadBach 1 0 DangerouslyUnstable 2 8 
[1, 1, 1, 1, 1, 1, 1, 2, 2, 0, 0, 0, 0, 0, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
[1, 1, 1, 1, 2, 0, 0, 0, 0, 2, 0, 2, 2, 0, 2, 2]


# some checking

In [5]:
base = '/nfs/nas-5.1/kyhuang/preprocess/cmv_raw_origin_v4/train//'

In [6]:
prepare_data, posts = [], []
for i, key in enumerate(['pos', 'neg']):
    with open(base+'/tree/pre_{}.jsons'.format(key)) as f:
        prepare_data.append([json.loads(_) for _ in f])
    with open(base+'/tree/clean_post_{}.jsons'.format(key)) as f:
        posts.append([json.loads(line) for line in f])

In [7]:
elmo_preprocess = []
for side, key in enumerate(['pos', 'neg']):
    with open(base+'/tree/cmv_elmo_{}.txt'.format(key)) as f:
        elmo_preprocess.append([line for line in f])

In [8]:
print(posts[0][0].keys())
print(posts[0][0]['content'][0].keys())

dict_keys(['content', 'topic', 'uid', 'pos_author'])
dict_keys(['bio', 'context', 'author', 'type'])


In [14]:
elmo_preprocess[0][1].split()[3:23]

['<ac>',
 'i',
 "'",
 've',
 'seen',
 'a',
 'lot',
 'of',
 'posts',
 'condemning',
 'islam',
 'as',
 'a',
 'violent',
 'religion',
 'or',
 'a',
 'sexist',
 'religion',
 '</ac>']

In [9]:
prepare_data[0][0]

{'topic_index': 0,
 'elmo_index': [1, 2, 3, 4, 5, 6, 7],
 'shell_span': [[0, 0, 2],
  [0, 23, 27],
  [0, 60, 63],
  [1, 0, 7],
  [1, 16, 17],
  [1, 40, 43],
  [3, 0, 0],
  [3, 18, 19],
  [3, 25, 27],
  [4, 0, 3],
  [4, 27, 29],
  [5, 0, 4],
  [6, 0, 0],
  [6, 17, 20],
  [6, 31, 33],
  [6, 44, 45]],
 'span': [[0, 3, 22],
  [0, 28, 59],
  [0, 64, 68],
  [1, 8, 16],
  [1, 18, 39],
  [1, 44, 68],
  [3, 1, 17],
  [3, 20, 25],
  [3, 28, 36],
  [4, 4, 26],
  [4, 30, 48],
  [5, 5, 31],
  [6, 1, 16],
  [6, 21, 30],
  [6, 34, 44],
  [6, 46, 51]],
 'mask': [[1, 1, 1, 1, 1, 1],
  [1, 1, 1, 1, 1, 1],
  [1, 1, 1, 1, 1, 1],
  [1, 1, 1, 1, 1, 1],
  [1, 1, 1, 1, 1, 1],
  [1, 1, 1, 1, 1, 1],
  [1, 1, 1, 1, 1, 1, 1, 1, 1],
  [0, 0, 0, 0, 0, 0, 1, 1, 1],
  [1, 1, 1, 1, 1, 1, 1, 1, 1],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1],
  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],


In [167]:
check = [[], []]
for side in range(2):
    for index, data in enumerate(posts[side]):
        temp = data['content'][1]['author']
        for _ in data['content']:
            if(_['author'] not in data['pos_author'] and _['author']!=temp):
                check[side].append(index)
                break
        

In [168]:
len(check[1]), len(posts[1])

(1119, 5635)

In [163]:
len(check[1]), len(posts[1])

(225, 1370)

In [158]:
for _ in data['content']:
    print(_['author'])

quietandproud
clairebones


In [143]:
count = [collections.defaultdict(int), collections.defaultdict(int)]
for side in range(2):
    for data in posts[side]:
        count[side][len(data['content'])] +=1
count

[defaultdict(int,
             {4: 1268,
              10: 169,
              2: 1002,
              6: 424,
              8: 169,
              7: 3,
              9: 1,
              5: 2,
              3: 1}),
 defaultdict(int, {4: 321, 2: 788, 6: 148, 8: 69, 3: 15, 9: 4, 5: 17, 7: 8})]

In [139]:
count = {}
for key in ['elmo_index', 'shell_span']:
    count[key] = [collections.defaultdict(int), collections.defaultdict(int)]
    
for side in range(2):
    for data in prepare_data[side]:
        for key in ['elmo_index', 'shell_span']:
            l = len(data[key])
            count[key][side][l] +=1
count

{'elmo_index': [defaultdict(int,
              {9: 189,
               17: 115,
               7: 187,
               20: 76,
               8: 170,
               5: 117,
               3: 34,
               18: 113,
               25: 39,
               23: 56,
               16: 130,
               15: 118,
               22: 62,
               11: 170,
               12: 171,
               6: 131,
               14: 154,
               13: 157,
               21: 72,
               19: 101,
               10: 174,
               62: 2,
               36: 16,
               31: 39,
               47: 4,
               45: 5,
               24: 52,
               29: 26,
               26: 50,
               30: 26,
               34: 19,
               4: 60,
               28: 25,
               2: 16,
               33: 16,
               51: 2,
               38: 7,
               32: 17,
               37: 10,
               35: 18,
               40: 8,
               27: 34,


In [135]:
side, index = 1, 1233
print(posts[side][index]['topic'])
print(posts[side][index]['pos_author'])
posts[side][index]['content']

i believe everyone's opinion deserves equal respect, even if i personally find their opinion ridiculous.
['Harlequnne', '[deleted]']


[{'bio': ['B I I I I E O B I I I I I E O B I I I I I I I I I I I I I I I I E O B I I I I I E O O B I I I I I I I I I E O',
   'B I I I I I I I I I I I I E O B I I I I I I I I I I I I I I I I I I E O O O O O O O O O O O O O O O O O O O O O O',
   'B I I I I I I I E O O B I I I I I I I I I I I I I I I I E O O O O O O O O O O O O O O O O',
   'O O O O O B I I I I I I I I I I I I I I I I I I E O O O O O'],
  'context': ["people hold lots of different views . it ' s part of being individuals . respect for those different views is , to me , one of the cornerstones of a civilized society . we are never all going to agree , but that does not mean we ca n ' t be respectful .",
   'i \' ve noticed a trend that , to me , is very disturbing . many of my friends , including my fiance , find it completely acceptable to mock those with a differing viewpoint . " how else will they learn ? " they say , which sounds like a cop - out to me .',
   'this is most specifically prevalent among christians and 

In [90]:
post.keys()

dict_keys(['content', 'topic', 'uid', 'pos_author'])

In [129]:
c = []
for side in range(2):
    for index, post in enumerate(posts[side]):
        if(('[deleted]' in post['pos_author']) and post['content'][-1]['author'] == '[deleted]'):
            c.append((side, index, len(post['content'])))
        elif('[deleted]' in post['pos_author']):
            c.append((side, index, len(post['content'])))
c

[(1, 48, 2),
 (1, 96, 2),
 (1, 126, 7),
 (1, 157, 2),
 (1, 355, 2),
 (1, 938, 2),
 (1, 959, 2),
 (1, 989, 8),
 (1, 1233, 3),
 (1, 1457, 5),
 (1, 1500, 4),
 (1, 1534, 2),
 (1, 1758, 5),
 (1, 1806, 2),
 (1, 1955, 2),
 (1, 1977, 2),
 (1, 2024, 2),
 (1, 2076, 6),
 (1, 2129, 2),
 (1, 2263, 4),
 (1, 2284, 8),
 (1, 2326, 2),
 (1, 2367, 2),
 (1, 2427, 5),
 (1, 2455, 2),
 (1, 2522, 2),
 (1, 2528, 2),
 (1, 2585, 6),
 (1, 2615, 8),
 (1, 2712, 8),
 (1, 2727, 5),
 (1, 2757, 2),
 (1, 2769, 5),
 (1, 2837, 2),
 (1, 2886, 3),
 (1, 3123, 8),
 (1, 3145, 8),
 (1, 3196, 2),
 (1, 3240, 2),
 (1, 3242, 2),
 (1, 3329, 4),
 (1, 3397, 4),
 (1, 3417, 2),
 (1, 3607, 2),
 (1, 3673, 3),
 (1, 3799, 2),
 (1, 4016, 4),
 (1, 4344, 2),
 (1, 4382, 8),
 (1, 4386, 2),
 (1, 4395, 2),
 (1, 4425, 4),
 (1, 4752, 6),
 (1, 4796, 8),
 (1, 4820, 2),
 (1, 4833, 6),
 (1, 4865, 4),
 (1, 4984, 9),
 (1, 5122, 9),
 (1, 5178, 2),
 (1, 5261, 2),
 (1, 5293, 2)]

In [120]:
for side in range(2):
    for index, data in enumerate(prepare_data[side]):
        if((len(data['para_pos_author']) == sum(data['para_pos_author'])) or sum(data['para_pos_author'])==0):
            print(len(posts[side][index]['content']))
            print(side, data['uid'], index)
            print(data['para_author'], len(data['para_author']))
            print(data['elmo_index'])
            

In [54]:
cmv_text[0][5371]

"<para> as an aside , <ac> i would caution against evaluating concepts like purpose or meaning from a single person ' s conscious perspective </ac> . certainly <ac> we ' ve learned enough to discredit many philosophical ideas of the past , like randomly believing in an afterlife or blindly following churches and holy books </ac> . the undeniable fact remains that <ac> we do not understand everything </ac> . <ac> is it not possible that the experience we have as individuals is part of a greater collective that we don ' t have the capacity to understand </ac> ? </para>\n"

In [45]:
pairs[236]['op_info']

{'bio': [None, None],
 'context': ["while i am not at all for the scare mongering and paranoia , i simply think that there is not enough evidence to show that gmos are completely safe to completely accept them with no questions asked . it seems very suspicious to me that large companies are lobbying so hard to avoid having to label gmos , as i do n't see why they would put so much time or money into fighting it unless there was something they were trying to hide . i have always tended to be wary of large corporations and their practices , but i feel like my current view is riddled with ignorance and misinformation . change my view ?",
  '<cite>'],
 'author': 'MagicSpaceMan',
 'type': [None, None],
 'topic': 'gmos are a relatively new development, and should be labeled and treated with caution',
 'pos_author': [[['MagicSpaceMan'], ['MagicSpaceMan']],
  [['MagicSpaceMan', 'MagicSpaceMan'], ['MagicSpaceMan', 'MagicSpaceMan']]]}

In [47]:
pairs[236]['content'][1][0]

[{'bio': [None, None, None],
  'context': ["so what do you mean by safety ? to consume ? this is really easy to determine . the chemicals which constitute food plants are really well known and we can test a plant for anything we want . so if you want to know if a food is safe to eat , send it to some chemists , and they 'll tell you if it has any dangerous concentrations of chemicals .",
   "you don't need to do a complex wait and see approach to it . you just use mass spectrometry. <link> there are other tests too , spectrometry is just the best at catching even traces .",
   'as far as labeling , they fight it because people would buy less if they saw the label .'],
  'author': 'huadpe',
  'type': [None, None, None]}]

In [32]:
data

{'topic_index': 5369,
 'elmo_index': [5370, 5371, 5372, 5373, 5374, 5375, 5376, 5377],
 'shell_span': [[0, 0, 0],
  [0, 15, 15],
  [0, 30, 33],
  [1, 0, 0],
  [1, 25, 30],
  [1, 50, 50],
  [2, 0, 0],
  [2, 34, 35],
  [2, 66, 68],
  [3, 0, 2],
  [3, 17, 18],
  [3, 36, 38],
  [4, 0, 1],
  [4, 16, 18],
  [5, 0, 4],
  [5, 25, 26],
  [5, 38, 38],
  [5, 56, 59],
  [5, 119, 119],
  [6, 0, 4],
  [6, 23, 23],
  [6, 74, 76],
  [6, 99, 99],
  [7, 0, 0],
  [7, 10, 10]],
 'span': [[0, 1, 14],
  [0, 16, 30],
  [0, 34, 42],
  [1, 1, 25],
  [1, 31, 49],
  [1, 51, 84],
  [2, 1, 33],
  [2, 36, 65],
  [2, 69, 77],
  [3, 3, 16],
  [3, 19, 36],
  [3, 39, 46],
  [4, 2, 16],
  [4, 19, 32],
  [5, 5, 25],
  [5, 27, 37],
  [5, 39, 55],
  [5, 60, 118],
  [5, 120, 136],
  [6, 5, 22],
  [6, 24, 74],
  [6, 77, 98],
  [6, 100, 125],
  [7, 1, 9],
  [7, 11, 19]],
 'mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
  [1, 1, 1,

In [26]:
int(2>1)

1

In [27]:
for side in range(2):
    c = [0, 0]
    for key, val in count['elmo_index'][side].items():
        c[int(key>81)] += val
    print(c)

[14120, 27]
[5629, 6]


## bert tree version

In [158]:
# load posts if needed
posts = []
for i, key in enumerate(['pos', 'neg']):
    with open('./../preprocess/cmv_raw_origin_v2/heldout/tree/clean_post_{}.jsons'.format(key)) as f:
        posts.append([json.loads(line) for line in f])

In [159]:
posts[0][0].keys()

dict_keys(['content', 'topic', 'uid', 'pos_author'])

In [179]:
# process link 
def find_shell_bert(index, last, context, adu_index):
    if(index==0):
        return (adu_index, 0, 0)
    for i in range(index, last, -1):
        if(context[i] in [".", "!", "?", "</ac>", "<para>"]):
            if(i>=index):
                return (adu_index, index, index)
            else:
                return (adu_index, i+1, index)
    
    return (adu_index, index, index)

def prepare_bert(data):
    mask = []
    shell_span, span, text = [], [], []
    adu_label = []
    author = []
    para_author = []
    topic = data['topic']
    
    ac_position_info = []
    adu_index = 0
    for post_pos, post in enumerate(data['content']):
        for bio, context in zip(post['bio'], post['context']):
            text.append( context.split() )

            bios, sent = bio.split(), context.split()

            last = -1
            for index, (bio, word) in enumerate(zip(bios, sent)):
                if(bio == 'B'):
                    span.append([adu_index, index, 0])
                    shell_span.append(find_shell_bert(index-1, last, text[-1], adu_index))
                
                if(bio == 'E'):
                    span[-1][-1] = index
                    last = span[-1][-1]
            
            text[-1] = ' '.join(text[-1])
            adu_index += 1
        
        # build mask
        dtype = post['type']
        prev_len, now_len = len(adu_label), len(dtype)
        for adu_pos, _ in enumerate(dtype):
            if(_ == 'P'):
                # constrain search space to this reply
                mask.append([0]*prev_len + [1]*now_len)

            elif(_ == 'C'):
                # constrain search space to all
                mask.append([1]*(prev_len + now_len))

            ac_position_info.append([adu_pos, post_pos])

        adu_label.extend( dtype )

    if(('[deleted]' in data['pos_author'][0]) and data['content'][-1]['author'] == '[deleted]'):
        for index, post in enumerate(data['content']):
            author.extend([1+index&1]*len(post['type']))
            para_author.extend([1+index&1]*len(post['bio']))
    else:
        for post in data['content']:
            if(post['author'] in data['pos_author'][0]):
                author.extend([1]*len(post['type']))
                para_author.extend([1]*len(post['bio']))
            elif(post['author'] in data['pos_author'][1]):
                author.extend([2]*len(post['type']))
                para_author.extend([2]*len(post['bio']))
            else:
                author.extend([0]*len(post['type']))
                para_author.extend([0]*len(post['bio']))
    adu_map = {'P':0, 'C':1}
    adu_label = [ adu_map[_] for _ in adu_label] 
        
    return {
        'topic':topic, 
        'text':text,
        'shell_span':shell_span, 
        'span':span, 
        'mask':mask, 
        'author':author,
        'para_author':para_author,
        'adu_label':adu_label,
        'ac_position_info':ac_position_info
    }

In [180]:
prepare_data = [[], []]

for side in range(2):
    for _ in posts[side]:
        temp = prepare_bert(_)
        temp['uid'] = _['uid']
        
        prepare_data[side].append(temp)

In [78]:
for i, key in enumerate(['pos', 'neg']):
    with open('./../preprocess/cmv_raw_origin_v2/heldout/tree_bert/pre_{}.jsons'.format(key), 'w') as f:
        for _ in prepare_data[i]:
            f.write(json.dumps(_))
            f.write('\n')